#### Imports

In [139]:
# Load in Libraries to read in a fasta file
from Bio import SeqIO
import sys
import numpy as np
import pandas as pd

#### Read in the File

In [152]:
# Read in "dna.example.fasta" file, in current directory
sequence = SeqIO.parse("dna2.fasta", "fasta")
sequence

#### Count the number of records in file

In [153]:
# Print out how many sequences are in the file
count = len(list(sequence))
print("There are", count, "sequences in the file")

There are 18 sequences in the file


#### List of Nucleotide Lengths

In [154]:
# Create an empty dataframe to store sequence information:
# Sequence ID, Sequence Length, GC Content
df = pd.DataFrame(columns=['Sequence', 'Sequence ID', 'Sequence Length', 'GC Content'])

for seq_record in SeqIO.parse("dna.example.fasta", "fasta"):
    # Get Sequence
    sequence1 = str(seq_record.seq)
    # Get the sequence ID
    seq_id = seq_record.id
    # Get the sequence length
    seq_length = len(seq_record.seq)
    # Get the GC content
    gc_content = (seq_record.seq.count("G") + seq_record.seq.count("C")) / seq_length * 100
    # Add the information to the dataframe
    df = pd.concat([df, pd.DataFrame({'Sequence': [sequence1],'Sequence ID': [seq_id], 'Sequence Length': [seq_length], 'GC Content': [gc_content]})], ignore_index=True)

C:\Users\Brendito\AppData\Local\Temp\ipykernel_11136\842254481.py:15: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, pd.DataFrame({'Sequence': [sequence1],'Sequence ID': [seq_id], 'Sequence Length': [seq_length], 'GC Content': [gc_content]})], ignore_index=True)


In [155]:
# Print out Head
print(df.head())

                                            Sequence  \
0  TCGGGCGAAGGCGGCAGCAAGTCGTCCACGCGCAGCGCGGCACCGC...   
1  ATTGGGGAGGAGGCGAGTTGAGCGGCGGCAGTTCGCCTGCGTGCGC...   
2  GACCTTGCATCGGCTGATCGCCGAGCGTGCCGACGTATTCATTCAC...   
3  GCCCGGCGATTTGACGTCTGCAGCCTCACGTCCAAACTCAATTTGA...   
4  GATCAGCCCCGCATACGCGTACGCGCGTGCGACGCCGAAGAGCGTC...   

                      Sequence ID Sequence Length  GC Content  
0   gi|142022655|gb|EQ086233.1|43             990   60.606061  
1  gi|142022655|gb|EQ086233.1|160             724   60.635359  
2   gi|142022655|gb|EQ086233.1|41            3080   61.428571  
3  gi|142022655|gb|EQ086233.1|221            2863   56.514146  
4  gi|142022655|gb|EQ086233.1|294            3832   67.327766  


In [156]:
# Print out the Minimum and the Maximum Sequence Length
print("The minimum sequence length is:", df['Sequence Length'].min())
print("The maximum sequence length is:", df['Sequence Length'].max())

The minimum sequence length is: 512
The maximum sequence length is: 4805


#### Open Reading Frames (ORFs)

In [157]:
def find_orfs(sequence, frame):
    start_codon = "ATG"
    stop_codons = {"TAA", "TAG", "TGA"}
    orfs = []
    seq_len = len(sequence)
    
    for i in range(frame - 1, seq_len, 3):
        codon = sequence[i:i+3]
        if codon == start_codon:
            for j in range(i, seq_len, 3):
                stop_codon = sequence[j:j+3]
                if stop_codon in stop_codons:
                    orfs.append((i, j + 3))
                    break
    return orfs

In [158]:
def longest_orf(orfs):
    if not orfs:
        return None
    return max(orfs, key=lambda x: x[1] - x[0])

In [174]:
longest_orf_overall = None
longest_orf_sequence_id = None

# Loop through each sequence in the dataframe
for index, row in df.iterrows():
    sequence = row['Sequence']
    sequence_id = row['Sequence ID']
    
    frame = 7
    
    orfs = find_orfs(sequence, frame)
    longest_orf_current = longest_orf(orfs)
    
    if longest_orf_current:
        if not longest_orf_overall or longest_orf_current[1] - longest_orf_current[0] > longest_orf_overall[1] - longest_orf_overall[0]:
            longest_orf_overall = longest_orf_current
            longest_orf_sequence_id = sequence_id

if longest_orf_overall:
    print(f"Longest ORF length: {longest_orf_overall[1] - longest_orf_overall[0]}")
    print(f"Sequence ID with longest ORF: {longest_orf_sequence_id}")
    print(f"Starting position of longest ORF: {longest_orf_overall[0] + 1}")
else:
    print("No ORFs found.")

Longest ORF length: 1686
Sequence ID with longest ORF: gi|142022655|gb|EQ086233.1|323
Starting position of longest ORF: 2824


#### Most Common Substrings

In [160]:
def find_repeats(sequence, length, current_df):
    # Create a dictionary to store the repeats and their counts
    repeats = {}
    
    # Loop through the sequence, looking for repeats of the specified length and adding them to the repeats.
    # If it already exists in the dictionary, increment the count.
    for i in range(len(sequence) - length + 1):
        repeat = sequence[i:i+length]
        if repeat in repeats:
            repeats[repeat] += 1
        else:
            repeats[repeat] = 1
    
    # Loop through the repeats dictionary and add the information to the dataframe.
    # If it already exists in the dataframe, increment the count by the count in the repeats dictionary.
    for repeat, count in repeats.items():
        if current_df.empty:
            current_df = pd.DataFrame({'Repeat': [repeat], 'Count': [count]})
        else:
            if repeat in current_df['Repeat'].values:
                current_df.loc[current_df['Repeat'] == repeat, 'Count'] += count
            else:
                current_df = pd.concat([current_df, pd.DataFrame({'Repeat': [repeat], 'Count': [count]})], ignore_index=True)
                
    return current_df

In [178]:
# Create a dataframe that is passed to the function for repeated substrings
# Should record combinations of nucleotides, IE ATC, ACG, etc. (depending on length of substring)
substrings = pd.DataFrame(columns=['Sequence', 'Count'])

# Frame for the repeated substrings
frame2 = 12

# Loop through each sequence in the dataframe
for index, row in df.iterrows():
    sequence = row['Sequence']
    sequence_id = row['Sequence ID']
    
    if frame2 != 0:
        print("Sequence ID:", sequence_id)
        substrings = find_repeats(sequence, frame2, substrings)
    else:
        for length in range(3, 7):
            # Print out the Sequence # and the Length of the Substring
            print("Sequence #:", index + 1, "Length of Substring:", length)
            
            # Find the repeated substrings
            substrings = find_repeats(sequence, length, substrings)

Sequence ID: gi|142022655|gb|EQ086233.1|43
Sequence ID: gi|142022655|gb|EQ086233.1|160
Sequence ID: gi|142022655|gb|EQ086233.1|41
Sequence ID: gi|142022655|gb|EQ086233.1|221
Sequence ID: gi|142022655|gb|EQ086233.1|294
Sequence ID: gi|142022655|gb|EQ086233.1|323
Sequence ID: gi|142022655|gb|EQ086233.1|564
Sequence ID: gi|142022655|gb|EQ086233.1|521
Sequence ID: gi|142022655|gb|EQ086233.1|455
Sequence ID: gi|142022655|gb|EQ086233.1|229
Sequence ID: gi|142022655|gb|EQ086233.1|422
Sequence ID: gi|142022655|gb|EQ086233.1|384
Sequence ID: gi|142022655|gb|EQ086233.1|280
Sequence ID: gi|142022655|gb|EQ086233.1|158
Sequence ID: gi|142022655|gb|EQ086233.1|59
Sequence ID: gi|142022655|gb|EQ086233.1|319
Sequence ID: gi|142022655|gb|EQ086233.1|438
Sequence ID: gi|142022655|gb|EQ086233.1|210
Sequence ID: gi|142022655|gb|EQ086233.1|237
Sequence ID: gi|142022655|gb|EQ086233.1|507
Sequence ID: gi|142022655|gb|EQ086233.1|350
Sequence ID: gi|142022655|gb|EQ086233.1|245
Sequence ID: gi|142022655|gb|EQ0862

In [176]:
# Order the dataframe by the count of the repeated substrings
substrings = substrings.sort_values(by='Count', ascending=False)
substrings

,Repeat,Count
160,CGGCGC,167
994,CGCGCG,149
159,GCGGCG,144
1473,GCCGGC,141
402,GCGCGC,141
...,...,...
3544,GCTTAG,1
3540,ACTTTA,1
3536,GAGTAA,1
3530,TGGAGT,1


In [177]:
# Print out the substring that was repeated the most. If there are multiple, print all of them.
max_count = substrings['Count'].max()
print("The substring(s) that was repeated the most is/are:")
print(substrings[substrings['Count'] == max_count])

# Do the same with minimum
# min_count = substrings['Count'].min()
# print("The substring(s) that was repeated the least is/are:")
# print(substrings[substrings['Count'] == min_count])

The substring(s) that was repeated the most is/are:
     Repeat  Count
160  CGGCGC    167
